In [1]:
# Install required dependencies
!pip install torch
!pip install tldextract
!pip install tensorflow


In [2]:
# Import required dependencies
import pandas as pd
import numpy as np
import tldextract
import json, os
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Activation
from keras.layers.embeddings import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
# Functions
def convert_json_to_df(f,file,plugin_type):
    data = json.load(f)
    features_plugin = data["titles"]
    if plugin_type=='vadinfo':
        features_plugin.remove('SHA256')
    plugin_df = pd.DataFrame(columns=features_plugin, data=data["data"])
    plugin_df['snapshot'] = int(file.split("/")[-2].split('-')[1])
    return plugin_df
def get_plugin_files(files):
    urls_df = pd.DataFrame()
    timestamp = ""
    for file in files:
        file_ext = file.split('/')[-1]
        with open(file,encoding='latin1') as f:
            try:
                urls_df_i = convert_json_to_df(f, file, "urls")
                urls_df = urls_df.append(urls_df_i)
                file_ext_dot = file_ext.split('.')
                file_ext_dot_dash = file_ext_dot[0].split("_")
                timestamp = file_ext_dot_dash[1]+"_"+file_ext_dot_dash[2]
            except:
                    print("error: "+file)
    return urls_df, timestamp
def get_domain(url):
    domain = tldextract.extract(url).domain
    if domain in ['ddns','bazar','onion']:
        url = url.strip('https://').strip('http://')
        urls = url.split('.')
        urls_i = urls.index(domain)
        if urls_i == 0:
            return domain
        return urls[urls_i-1]
    return domain
def get_domain_space(domain):
    try:
        return " ".join(domain)
    except:
        return ""
def argmin_label(row):
    emb = np.array(row[[0,1,2,3,4,5,6,7,8,9]])
    list_dist = [np.sum(np.power(emb-ref_families[key],2)) for key in ref_families]
    arg_m = np.argmin(list_dist)
    dist_m = np.min(list_dist)
    if dist_m < 0.5:
        row['predict_family_label'] = list(families)[arg_m]
    else:
        row['predict_family_label'] = 'others'
    row['predict_family_dist'] = dist_m
    return row
def predict_family_df(domain):
    output_df = pd.DataFrame(columns = ['char_'+str(i) for i in range(75)] , data = domain)
    y_pred = model.predict(domain)
    output_df["predict"] = y_pred
    domain_emb = model_family.predict(domain)
    output_df = pd.concat([output_df, pd.DataFrame(domain_emb)], axis=1)
    output_df = output_df.apply(lambda row : argmin_label(row),axis=1)
    output_df.drop([0,1,2,3,4,5,6,7,8,9], axis=1, inplace=True)
    return output_df

In [4]:
# Loading binary model
model = tf.keras.models.load_model('/raid0/haim/haim/dga_model_keras')

In [5]:
# Loading family model
model_family = tf.keras.models.load_model('/raid0/haim/haim/dga_family_model_keras')

In [6]:
# Loading reference embedding vectors
ref_save_df = pd.read_csv('model_ref_new.csv')
ref_save_df

,Family,0,1,2,3,4,5,6,7,8,9
0,emotet,0.200769,-0.237580,0.065239,0.457636,-0.276223,0.178562,-0.090978,0.248116,0.528811,-0.481559
1,banjori,0.307406,0.259920,-0.282230,-0.353246,-0.436052,0.637836,-0.155065,-0.103501,-0.026574,0.032663
2,rovnix,-0.382953,-0.217121,-0.079289,-0.648299,-0.215187,-0.178181,0.123139,-0.252936,0.414872,-0.224310
3,flubot,-0.340346,0.125373,0.097962,0.102049,0.066690,0.180567,-0.227522,-0.233104,-0.237740,-0.805456
4,gameover,0.084834,0.226634,0.181872,-0.067027,-0.312795,-0.473130,-0.573160,0.478703,-0.151817,-0.038204
5,qadars,-0.102191,0.589433,0.671858,-0.042507,-0.018336,0.232687,0.305588,0.021762,0.154993,0.128705
6,padcrypt,0.251891,0.530984,-0.469107,0.078098,0.018390,-0.188725,0.356227,0.311306,0.280558,-0.299947
7,ramdo,0.503298,-0.156416,0.000823,-0.129468,0.484918,-0.020062,-0.444824,-0.218678,0.435629,-0.185619
8,corebot,-0.017426,0.155469,-0.178655,0.172860,-0.369482,-0.254771,-0.015373,-0.831954,0.131788,-0.050511
9,goz,-0.541632,0.419992,-0.288840,0.107990,0.080512,0.108628,-0.529812,0.083337,0.255371,0.252999


In [7]:
ref_families = {}
for dga_family in ref_save_df['Family']:
    ref_families[dga_family] = np.array(ref_save_df[ref_save_df['Family']==dga_family][['0','1','2','3','4','5','6','7','8','9']])

In [8]:
families = list(ref_families.keys())

In [9]:
# Read tokenizer
tokenizer = Tokenizer()
tokenizer.word_index = pd.read_csv('tokenizer.csv').set_index('keys')['values'].to_dict()

In [10]:
# Read the URL plugins
path = "/raid0/haim/haim/URL_Snapshots/"
snapshots = os.listdir(path)
snapshots = [int(x.split('-')[1]) for x in snapshots if 'snap' in x]
snapshots.sort()
snapshots = ['snapshot-'+str(x) for x in snapshots]
for snap_file in snapshots:
    print(path+snap_file+"/")
    snap_file = path+snap_file+"/"
    files = []
    for r, d, f in os.walk(snap_file):
        for file in f:
            if '.json' in file:
                files.append(os.path.join(r, file))
    #try:
        data, timestamp = get_plugin_files(files)

/raid0/haim/haim/URL_Snapshots/snapshot-1/
error: /raid0/haim/haim/URL_Snapshots/snapshot-1/.ipynb_checkpoints/urls_2022-01-30_10-25-58.438453-checkpoint.json


In [11]:
data

,PID,Process,Heap,Virtual-address,URL,snapshot
0,9900,notepad.exe,0,0x15ceb795bd0,https://nwbqonnspiiwjrpo.cn,1
1,9900,notepad.exe,0,0x15ceb7d2fe8,https://ipajpheumymwwhlg.onion,1
2,9900,notepad.exe,0,0x15ceb7d3006,https://uifpby48sxnu97yn.ru,1
3,9900,notepad.exe,0,0x15ceb7d3024,https://m57mz3ocfrrx55ky.bazar,1
4,9900,notepad.exe,0,0x15ceb7d3042,https://bb7now8hgnwj1o42aoe1e6qzpc.com,1
5,9900,notepad.exe,0,0x15ceb7d308a,https://3vjnac1xj87dl1vauzom1bfuqzn.net,1
6,9900,notepad.exe,0,0x15ceb7d30c2,https://google.com,1
7,9900,notepad.exe,0,0x15ceb7d30fa,https://faceebook-com.bugs3.com/login/Secured_...,1
8,9900,notepad.exe,0,0x15ceb7d3132,https://nvidia.com,1
9,9900,notepad.exe,0,0x15ceb883bea,https://microsoft.com,1


In [12]:
data['Domain'] = data['URL'].apply(get_domain)

In [13]:
data_chars = data['Domain'].apply(get_domain_space)

In [14]:
data_tokens = tokenizer.texts_to_sequences(data_chars)
data_tokens = pad_sequences(data_tokens, maxlen=75, padding='post')

In [15]:
data_pred = predict_family_df(data_tokens)

In [16]:
data_pred

,char_0,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,...,char_68,char_69,char_70,char_71,char_72,char_73,char_74,predict,predict_family_label,predict_family_dist
0,7.0,21.0,17.0,24.0,4.0,7.0,7.0,5.0,14.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.992725e-01,emotet,0.210194
1,3.0,14.0,2.0,25.0,14.0,15.0,1.0,12.0,11.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.991703e-01,emotet,0.194278
2,12.0,3.0,20.0,14.0,17.0,19.0,29.0,32.0,5.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.982139e-01,chinad,0.014983
3,11.0,34.0,35.0,11.0,26.0,30.0,4.0,10.0,20.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.982387e-01,gspy,0.009246
4,17.0,17.0,35.0,7.0,4.0,21.0,32.0,15.0,16.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.990136e-01,gameover,0.013017
5,30.0,22.0,25.0,7.0,2.0,10.0,27.0,23.0,25.0,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.960905e-01,gameover,0.016691
6,16.0,4.0,4.0,16.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.787902e-07,others,1.842959
7,17.0,12.0,16.0,5.0,30.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.735253e-06,others,1.731859
8,7.0,22.0,3.0,13.0,3.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.687827e-06,others,1.893812
9,11.0,3.0,10.0,6.0,4.0,5.0,4.0,20.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.889452e-05,others,1.870149


In [17]:
# Create onnx models

In [25]:
! pip install onnxruntime

In [26]:
! pip install git+https://github.com/onnx/tensorflow-onnx

  Cloning https://github.com/onnx/tensorflow-onnx to /tmp/pip-req-build-m4gbp_x6
  Created wheel for tf2onnx: filename=tf2onnx-1.10.0-py3-none-any.whl size=439728 sha256=c28fa58a365488301346db45fc3f39c41e65d86295818351446c4f4af1389e79
  Stored in directory: /tmp/pip-ephem-wheel-cache-kz8egioe/wheels/8d/c7/fe/7dc73855334ce8bd94435f3c357aeca17c69252864204cb133
Successfully built tf2onnx


In [27]:
!python -m tf2onnx.convert --saved-model /raid0/haim/haim/dga_model_keras --output dga_binary_model_tensorflow.onnx

2022-03-21 11:43:08.843689: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-21 11:43:08.843743: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/opt/conda/lib/python3.8/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-03-21 11:43:10.236918: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-21 11:43:10.236962: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-21 11:43:10.23

In [57]:
!python -m tf2onnx.convert --saved-model /raid0/haim/haim/dga_family_model_keras --output dga_family_model_tensorflow.onnx

2022-03-21 12:50:15.972632: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-21 12:50:15.972684: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/opt/conda/lib/python3.8/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-03-21 12:50:17.731481: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-21 12:50:17.731544: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-21 12:50:17.73

In [ ]:
# Inference onnx model

In [28]:
import onnx
import onnxruntime

In [29]:
ONNX_DGA_FILE_PATH = "/raid0/haim/haim/dga_binary_model_tensorflow.onnx"

In [31]:
session = onnxruntime.InferenceSession(ONNX_DGA_FILE_PATH, None)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name



In [32]:

result = session.run([output_name], {input_name: data_tokens.astype(np.float32)})


In [33]:
result

[array([[9.9927253e-01],
        [9.9917036e-01],
        [9.9821395e-01],
        [9.9823874e-01],
        [9.9901360e-01],
        [9.9609041e-01],
        [8.3446503e-07],
        [1.6987324e-06],
        [3.6656857e-06],
        [4.8875809e-05]], dtype=float32)]

In [58]:
ONNX_DGA_FAMILY_FILE_PATH = "/raid0/haim/haim/dga_family_model_tensorflow.onnx"

In [59]:
session = onnxruntime.InferenceSession(ONNX_DGA_FAMILY_FILE_PATH, None)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name


In [60]:
result = session.run([output_name], {input_name: data_tokens.astype(np.float32)})

In [65]:
result[0].shape

(10, 10)